In [13]:
import pickle
import scipy.sparse as sp
# Saving the list of IDs as a serialized object
with open('users.pkl', 'wb') as f:
    pickle.dump(users, f)



In [14]:
# Loading the list of IDs from the serialized object
with open('users.pkl', 'rb') as f:
    id_list = pickle.load(f)

In [16]:
import pandas as pd

# Saving the dataframe as a CSV file
main.to_csv('main.csv', index=False)

In [17]:
# Loading the dataframe from the CSV file
df = pd.read_csv('main.csv')

In [5]:
import scipy.sparse as sp
# Save the matrix to a file
sp.save_npz('matrix.npz', user_mat)

In [3]:
# Load the matrix from a file
matrix = sp.load_npz('matrix.npz')

In [ ]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [ ]:
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")
RUN_ID='fa350b60cf564ff1a1d3341e2ac38cbe'
#logged_model = f'mlflow-artifacts:/728115650857226939/eb5ec70b031246f3a1336ba5ea0afe76/artifacts/model'
logged_model=f'mlflow-artifacts:/773732190913986377/{RUN_ID}/artifacts/model'
#logged_model = f'runs:/{RUN_ID}/model'
model = mlflow.sklearn.load_model(logged_model)
from flask import Flask, request, jsonify
import data_helper

main=data_helper.main
users=data_helper.users
def recommender(user_id, data=data_helper.user_mat, model=model):
    model.fit(data)
    index = users.index(user_id)
    current_user = main[main['user_id']==user_id]
    distances, indices = model.kneighbors(data[index], 15)
    recomendation = []
    for i in indices[0]:
        user = main[main['user_id']==users[i]]
        for i in user['category'].unique():
            if i not in current_user['category'].unique():
                recomendation.append(i)
    return recomendation
#     print(indices)
#print(recommender('5df49b32cc709107827fb3c7')[:10])
#recommender(users[0])[:10]
app = Flask('recommender-prediction')


@app.route('/predict', methods=['POST'])
def predict_endpoint():
    rec = request.get_json()

    pred = recommender(rec)[:10]

    result = {
        "id":rec,
        'recommendations': pred,
        'model_version': RUN_ID
    }

    return jsonify(result)


if __name__ == "__main__":
    app.run(debug=True, host='0.0.0.0', port=9696)

In [25]:
import model
actual_result = model.base64_decode(base64_input)


AttributeError: module 'model' has no attribute 'base64_decode'

In [37]:
def recommender(user_id, data=matrix, model=model):
    model.fit(data)
    index = users.index(user_id)
    current_user = main[main['user_id']==user_id]
    distances, indices = model.kneighbors(data[index], 15)
    recomendation = []
    for i in indices[0]:
        user = main[main['user_id']==users[i]]
        for i in user['category'].unique():
            if i not in current_user['category'].unique():
                recomendation.append(i)
    return recomendation

NameError: name 'matrix' is not defined